# Lesson 6. Replicating simulations

### SA421 Fall 2015

* As usual, we begin with some setup code. 


* Note that we are now importing:
    - the `mean`, `std` and `sqrt` functions from `numpy` and
    - the `t` random variable object from `scipy.stats`.

In [1]:
##### Setup #####
# Import everything from SimPy
from SimPy.Simulation import *

# Import mean, std, sqrt functions from NumPy
from numpy import mean, std, sqrt

# Import seed initializer and random sampling functions from NumPy
from numpy.random import seed, uniform, exponential

# Import t random variable from scipy.stats
from scipy.stats import t

* Here's the SimPy simulation code for the original Fantastic Dan problem, with a monitor for the customer delay (i.e. time in queue).


* The `print` statements have been omitted, since we will be running the simulation many times.

In [2]:
##### Parameters #####
class P:
    # Customers arrive at the entrance with exponentially distributed
    # interarrival times with mean 20
    interarrivalTimeMean = 20
    
    # Service times are uniformly distributed between 15 and 25
    serviceTimeMin = 15
    serviceTimeMax = 25
    
    # One server: Fantastic Dan works by himself
    nServers = 1
    
    # Shop is open for 6 continuous hours
    simulationTimeMax =  6 * 60
    

##### Processes #####
# Customer
class Customer(Process):
    def behavior(self):
        # Customer arrives, joins queue
        # Record customer arrival time
        arrivalTime = now()
        yield request, self, R.server
        
        # Customer is released from queue and starts service
        # Observe customer delay using monitor
        M.delay.observe(now() - arrivalTime)
        serviceTime = uniform(low = P.serviceTimeMin, high = P.serviceTimeMax)
        yield hold, self, serviceTime
        
        # Customer finishes service, leaves
        yield release, self, R.server

# Entrance
class Entrance(Process):
    def behavior(self):
        # At the start of the simulation, no customers have arrived
        nCustomers = 0
        
        # Customer arrivals
        while True:
            # Wait until the next arrival
            interarrivalTime = exponential(scale = P.interarrivalTimeMean)
            yield hold, self, interarrivalTime
            
            # Create a new customer using the template defined in the Customer class
            c = Customer(name="Customer {0}".format(nCustomers))
            
            # Activate the customer's behavior
            activate(c, c.behavior())

            # Count this new customer
            nCustomers += 1

            
##### Resources #####
class R:
    # Server
    server = None
    
    
##### Monitors #####
class M:
    # Delay
    delay = None


##### Model #####
def model():
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator
    seed(123)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)
    
    # Create the monitor for customer delay
    M.delay = Monitor()

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

* Instead of running the simulation just once, let's run it 50 times.


* This way, we can get a better idea of what's going on at Fantastic Dan's shop when we sample other values for the interarrival and service times. 


* Note that the simulation output is always the same for a given seed! To get different outputs, we need to use different seeds.


* We also need to be able to record the output from the different simulation replications.


* To do this, let's modify the `model()` function to take a seed as input, and output the average delay. 

In [3]:
##### Model #####
def model(inputSeed):
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create the server resource
    R.server = Resource(capacity = P.nServers)
    
    # Create the monitor for customer delay
    M.delay = Monitor()

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

    # Compute performance measure: average delay
    avgDelay = M.delay.mean()
    
    # Return performance measure
    return avgDelay

* Now we're ready to replicate the simulation 50 times.


* As seeds, let's just use multiples of 123 (i.e. 0, 123, 246, 369, ...)

In [4]:
##### Experiment #####
# Number of replications
n = 50

# Perform replications, and put average delay from each replication in one list
avgDelayObs = [model(inputSeed = 123*i) for i in range(n)]

# Print the list for examination
print("Observed average delay from all replications = {0}".format(avgDelayObs))

Observed average delay from all replications = [3.6789183539185712, 35.261587818047126, 16.87216575017206, 39.60902545745712, 46.17568216201532, 9.933433342891021, 8.466797143568746, 45.86623774393007, 63.21204718783098, 36.92265688539873, 20.41607502364714, 58.54995123685904, 13.795485558314104, 5.428505224453253, 20.039887068372174, 9.503698863331275, 52.23978364090359, 30.46999467869552, 16.920071942447787, 42.84188572294847, 12.063269371855252, 25.19124205356353, 9.142984722053724, 44.378396120184775, 55.58713779652265, 6.0413235018044364, 13.99359505440943, 42.01433775629554, 6.082691276264449, 50.82460696458097, 34.20221342593574, 65.53890582155756, 21.83662173569419, 27.193495296775104, 20.03374638100023, 31.59505860816319, 10.644622172958808, 23.098694529918898, 27.118307660150265, 30.140525229145407, 15.392623548149215, 39.06769687166434, 46.734325352516905, 40.35774803390669, 24.239986640788867, 53.00366333832998, 18.002896041837214, 9.409819885837821, 26.321378391252914, 24.

* Now that we've gathered the average delay from 50 replications, we can compute the sample mean, sample standard deviation, and confidence intervals to estimate the true mean average delay.


* `mean(x)` simply computes the arithmetic mean of the items in list `x`. [(documentation)](http://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html)


* `std(x, ddof=k)` computes the standard deviation of the items in list `x`, with divisor $n - k$, where $n$ is the number of elements in `x`. [(documentation)](http://docs.scipy.org/doc/numpy/reference/generated/numpy.std.html)
    - We will almost always use `ddof = 1`.


* `t` is a random variable object from `scipy.stats` [(documentation)](http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html)
    - A random variable object comes with many methods, such as probability density functions, cumulative density functions, expected value, variance, etc.
    - `t.ppf(q, df)` gives the value of the $(100 \times q)$-th percentile for the $t$-distribution with $df$ degrees of freedom.   

In [5]:
# Observed sample mean
avgDelaySM = mean(avgDelayObs)

# Observed sample standard deviation
avgDelaySSD = std(avgDelayObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
avgDelayCIL = avgDelaySM - t.ppf(1 - alpha/2, n - 1) * avgDelaySSD / sqrt(n)
avgDelayCIR = avgDelaySM + t.ppf(1 - alpha/2, n - 1) * avgDelaySSD / sqrt(n)

print("Average delay:")
print("  Sample mean: {0}".format(avgDelaySM))
print("  Sample standard deviation: {0}".format(avgDelaySSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, avgDelayCIL, avgDelayCIR))

Average delay:
  Sample mean: 28.596857730761734
  Sample standard deviation: 16.847604839684355
  95.0% confidence interval: [23.808821418978965, 33.3848940425445]


## With a neighbor...

Use 50 replications to obtain a point estimate (i.e. observed sample mean) and an interval estimate (i.e. confidence interval) of the time average number of customers in the queue for the Fantastic Dan problem. Give an interpretation of your confidence interval.

Let's start by modifying the `model()` function so that it

* activates the built-in monitors for `R.server` (see `#1`)
* computes and outputs the time average number of customers in the queue using `R.server.waitMon`

All the other aspects of the model (i.e. parameters, processes, resources, etc.) can stay the same.

In [6]:
##### Model #####
def model(inputSeed):
    # Initialize SimPy 
    initialize()

    # Initialize a seed for the random number generator
    seed(inputSeed)

    # Create the server resource
    R.server = Resource(capacity = P.nServers, monitored = True)    #1
    
    # Create the monitor for customer delay
    M.delay = Monitor()

    # Activate the entrance (to generate customers)
    e = Entrance()
    activate(e, e.behavior())
    
    # Run the simulation
    simulate(until = P.simulationTimeMax)

    # Compute performance measure: time average number of customers in the queue
    tAvgCustQ = R.server.waitMon.timeAverage()
    
    # Return performance measure
    return tAvgCustQ

Now, let's perform 50 replications of the simulation and record the time average number of customers in the queue for each replication.

Instead of using a list comprehension like above, we can build a list using the `.append()` method.

In [7]:
##### Experiment #####
# Number of replications
n = 50

# Perform replications, and put time average number of customers
# from each replication in one list
tAvgCustQObs = []
for i in range(n):
    tAvgCustQObs.append(model(inputSeed = 123*i))

# Print the list for examination
print("Observed time average number of customers in the queue from all replications = {0}".format(tAvgCustQObs))

Observed time average number of customers in the queue from all replications = [0.14306904709683332, 2.068757607229012, 0.7030069062571691, 1.9908501697637524, 2.861249589035309, 0.3587073151599535, 0.3057454524066492, 2.446935944343226, 4.828090036161945, 1.9629602947946498, 0.8506697926519642, 3.7955031350364203, 0.4981703118280093, 0.3304961404756987, 0.8657313128359124, 0.36958828912954955, 3.2766523452901626, 1.5234997339347762, 0.7029414592759308, 2.4252226648745436, 0.50568260542766, 1.1533765967380325, 0.4571492361026862, 2.3919744650222032, 2.6458519004082226, 0.25172181257518483, 0.7573661927452396, 2.100716887814777, 0.27728654660518676, 3.461751298846052, 1.516339057639677, 4.109233620308379, 1.065098093069711, 1.359674764838755, 0.8577824266957933, 1.5765344849069876, 0.49908403315501065, 0.9193633873290052, 1.599549811291542, 1.8264394646045616, 0.6452322456094991, 2.524033228079168, 2.945311121673198, 2.1611071866090583, 1.0994418498076506, 3.3370146922090345, 0.93382966

With these observations in hand, we can compute point and interval estimates of the mean time average number of customers in the queue, using essentially the same code we wrote for average delay.

In [8]:
# Observed sample mean
tAvgCustQSM = mean(tAvgCustQObs)

# Observed sample standard deviation
tAvgCustQSSD = std(tAvgCustQObs, ddof = 1)

# Confidence level 0.05
alpha = 0.05
tAvgCustQCIL = tAvgCustQSM - t.ppf(1 - alpha/2, n - 1) * tAvgCustQSSD / sqrt(n)
tAvgCustQCIR = tAvgCustQSM + t.ppf(1 - alpha/2, n - 1) * tAvgCustQSSD / sqrt(n)

print("Time average number of customers in the queue:")
print("  Sample mean: {0}".format(tAvgCustQSM))
print("  Sample standard deviation: {0}".format(tAvgCustQSSD))
print("  {0}% confidence interval: [{1}, {2}]".format((1 - alpha)*100, tAvgCustQCIL, tAvgCustQCIR))

Time average number of customers in the queue:
  Sample mean: 1.5700461194582926
  Sample standard deviation: 1.1472859506589907
  95.0% confidence interval: [1.2439910603549253, 1.89610117856166]


We are 95% <span style="color:#a00000;">confident</span> that the true mean time average number of customers in the queue lies in the interval $[1.24, 1.90]$ (rounding the left endpoint down, rounding the right endpoint up): if we repeated this experiment many, many times, 95% of the resulting confidence intervals would contain the true mean time average number of customers in the queue.